In [66]:
import fractions
import math
from qiskit_aer import AerSimulator, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.quantum_info import Operator
import numpy as np


def classical_modular_exponentiation(a, power, N):
    """Performs classical modular exponentiation."""
    return pow(a, int(power), int(N))

def quantum_order_finding(a, N):
    """Performs the quantum part of the order-finding algorithm."""
    # Use Qiskit to construct the circuit for quantum order finding
    # This is a simplified version and might not work for all cases
    n = math.ceil(math.log2(N))
    qc = QuantumCircuit(2 * n, n)
    for q in range(n):
        qc.h(q)
    qc.x(n)
    for q in range(n):
        qc.append(Operator([[1, 0], [0, np.exp(2 * np.pi * 1j / (2 ** (q + 1)))]], input_dims=(2,)),
                  [q + n])
    qc.measure(range(n), range(n))
    
    # Transpile the circuit for the AerSimulator
    simulator = AerSimulator()
    transpiled_circuit = transpile(qc, simulator)
    
    # Execute the transpiled circuit
    job = simulator.run(transpiled_circuit, shots=1)
    result = job.result()
    measurements = list(result.get_counts().keys())[0]
    phase = int(measurements, 2) / (2 ** n)
    
    # Use classical methods to find the order from the phase
    frac = fractions.Fraction(phase).limit_denominator(N)
    r = frac.denominator
    return r

def shors_algorithm(N, max_attempts=100):
    """Performs Shor's algorithm to factorize N."""
    if N <= 2:
        return 1, 0  # Return (1, 0) for numbers less than or equal to 2
    
    attempt = 0
    while attempt < max_attempts:
        # Step 1: Choose a random number a < N
        a = np.random.randint(2, N)
        
        # Step 2: Compute the greatest common divisor of a and N
        gcd = math.gcd(a, N)
        if gcd > 1:
            # We found a non-trivial factor
            return gcd, N // gcd
        
        # Step 3: Use the quantum algorithm to find the order r of a modulo N
        r = quantum_order_finding(a, N)
        
        # Step 4: If r is odd or a^(r/2) is congruent to -1 mod N, go back to step 1
        if r % 2 != 0 or classical_modular_exponentiation(a, r // 2, N) == N - 1:
            attempt += 1
            continue
        
        # Step 5: Compute the factors of N
        factor1 = math.gcd(classical_modular_exponentiation(a, r // 2, N) + 1, N)
        factor2 = math.gcd(classical_modular_exponentiation(a, r // 2, N) - 1, N)
        
        # Check if the factors are non-trivial
        if factor1 != 1 and factor2 != 1:
            return factor1, factor2
        else:
            attempt += 1

    return 1, 0  # Return (1, 0) if the algorithm fails
    
def is_prime(n):
    """Checks if a number is prime."""
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def factor_flags(factor1, factor2):
    """Returns a binary flag based on the primality of the factors."""
    flag = ""
    if is_prime(factor1):
        flag += "1"
    else:
        flag += "0"
    if is_prime(factor2):
        flag += "1"
    else:
        flag += "0"
    return flag

# Example usage
N = 39
factors = shors_algorithm(N)
print(f"The factors of {N} are {factors}")
flag = factor_flags(*factors)
print(flag)
print(f"The binary flag for the factors is {flag}")


The factors of 39 are (3, 13)
11
The binary flag for the factors is 11


In [72]:
from qiskit.circuit.library import MCMT
from qiskit.circuit.library import GroverOperator
def oracle(circuit, flags, qubits):
    """Marks the states with both factors prime ('11')."""
    qubits_list = list(qubits)  # Convert qubits to a list
    for i, flag in enumerate(flags):
        if flag == '11':
            circuit.x(qubits_list[i])
    # Apply a multi-controlled Z gate to flip the phase of the marked states
    circuit.h(qubits_list[-1])
    circuit.mcx(qubits_list[:-1], qubits_list[-1])
    circuit.h(qubits_list[-1])
    for i, flag in enumerate(flags):
        if flag == '11':
            circuit.x(qubits_list[i])


def grovers_algorithm(flags, num_qubits, shots=10000):
    """Estimates the number of elements with both factors prime using Grover's algorithm."""
    # Create a quantum circuit
    qc = QuantumCircuit(num_qubits)

    # Apply Hadamard gates to all qubits
    qc.h(range(num_qubits))

    # Construct the oracle
    oracle_qc = QuantumCircuit(num_qubits, name='Oracle')
    oracle(oracle_qc, flags, range(num_qubits))
    oracle_gate = oracle_qc.to_gate()
    oracle_gate.num_ancillas = 0  # Manually set the number of ancillary qubits

    # Construct the Grover operator
    grover_op = GroverOperator(oracle_gate, insert_barriers=True)

    # Apply the Grover operator a certain number of times
    iterations = int(np.sqrt(2**num_qubits))
    for _ in range(iterations):
        qc.append(grover_op, range(num_qubits))

    # Measure the qubits
    qc.measure_all()

    # Transpile the circuit for the AerSimulator
    backend = Aer.get_backend('qasm_simulator')
    transpiled_circuit = transpile(qc, backend)

    # Execute the transpiled circuit
    job = backend.run(transpiled_circuit, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)

    # Estimate the number of solutions
    num_solutions = 0
    for state, count in counts.items():
        if state[::-1] == '11':  # Check if the measured state corresponds to '11'
            num_solutions += count

    return num_solutions / shots * 2**num_qubits

# Example usage
# Generate a list of 20 random numbers between 1 and 50
numbers = np.random.randint(1, 51, size=20)
print(numbers)
# Find the factors of each number and convert them into binary flags
flags = [factor_flags(*shors_algorithm(num)) for num in numbers]

# Create the oracle for Grover's algorithm
print(flags)
num_qubits = len(flags)
estimated_count = grovers_algorithm(flags, num_qubits)
print(f"Estimated number of numbers with both factors prime: {estimated_count}")

[34 25 41 21 14  1  7  2  1  6 23 43 30 15 12 45 24 49 22  3]
['11', '11', '00', '11', '11', '00', '00', '00', '00', '11', '00', '00', '01', '11', '10', '10', '10', '11', '11', '00']
